In [15]:
import os

def rename_folders(base_dir):
    iteration = 1  # Start iteration counter

    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(base_dir):
        for subdir in dirs:
            subdir_path = os.path.join(root, subdir)
            target_file = os.path.join(subdir_path, "2_720p.mkv")

            # Check if the target file exists in the subfolder
            if not os.path.exists(target_file):
                # Rename the subfolder
                new_name = f"no video+{iteration}"
                new_path = os.path.join(root, new_name)
                os.rename(subdir_path, new_path)
                print(f"Renamed: {subdir_path} -> {new_path}")
                iteration += 1

# Specify the base directory
base_directory = r"F:\AIM Lab\Experiment"  # Root folder
rename_folders(base_directory)

PermissionError: [WinError 5] Access is denied: 'F:\\AIM Lab\\Experiment\\2014-2015' -> 'F:\\AIM Lab\\Experiment\\no video+1'

In [ ]:
import json
import os
from moviepy import VideoFileClip, concatenate_videoclips

def extract_timestamps(json_file):
    """
    Extract timestamps for specific events from the JSON file.
    """
    # Load the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Define the labels to extract
    target_labels = {"Goal", "Penalty", "Red card", "Direct free-kick", "Shots on target"}
    timestamps = []

    # Extract timestamps for the target labels
    for annotation in data.get("annotations", []):
        if annotation["label"] in target_labels:
            game_time = annotation["gameTime"]
            minutes, seconds = map(int, game_time.split(" - ")[1].split(":"))
            start_time = max(0, (minutes * 60 + seconds) - 3)  # Start time (-3 seconds)
            end_time = (minutes * 60 + seconds) + 3  # End time (+3 seconds)
            timestamps.append((start_time, end_time))

    return timestamps

def trim_and_merge_video(video_path, timestamps, output_path):
    """
    Trim the video based on the extracted timestamps and merge the clips.
    """
    # Load the video
    video = VideoFileClip(video_path)
    clips = []

    # Trim the video based on timestamps
    for start, end in timestamps:
        clip = video.subclipped(start, end)
        clips.append(clip)

    # Concatenate all clips
    final_video = concatenate_videoclips(clips)

    # Write the output video
    final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Close the video objects
    video.close()
    for clip in clips:
        clip.close()

# Paths
base_folder = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley"
json_file = os.path.join(base_folder, "Labels-v2.json")
video_file = os.path.join(base_folder, "2_720p.mkv")
output_file = os.path.join(base_folder, "2_720pT.mkv")

# Extract timestamps and process the video
timestamps = extract_timestamps(json_file)
if os.path.exists(video_file):
    trim_and_merge_video(video_file, timestamps, output_file)
    print(f"Trimmed video saved as: {output_file}")
else:
    print(f"Video file not found: {video_file}")

MoviePy - Building video F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv.
MoviePy - Writing audio in 2_720pTTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv



MoviePy - Done !
MoviePy - video ready F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv
Trimmed video saved as: F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv


In [19]:
import cv2
import numpy as np
import os

def process_video(input_path, output_path):
    """
    Process the video to:
    1. Convert to grayscale.
    2. Remove grass and audience areas while preserving the ball.
    """
    # Load video
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Changed to mp4v for better compatibility
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)
    
    print(f"Processing {total_frames} frames...")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Convert original frame to HSV for better color detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define green grass color range in HSV (more specific range)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])
        grass_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Define ball detection (white/light colored ball)
        # Method 1: HSV-based ball detection
        lower_ball_hsv = np.array([0, 0, 200])
        upper_ball_hsv = np.array([180, 30, 255])
        ball_mask_hsv = cv2.inRange(hsv, lower_ball_hsv, upper_ball_hsv)
        
        # Method 2: Grayscale-based ball detection (bright pixels)
        _, ball_mask_gray = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_BINARY)
        
        # Combine both ball detection methods
        ball_mask = cv2.bitwise_or(ball_mask_hsv, ball_mask_gray)
        
        # Apply morphological operations to clean up the ball mask
        kernel = np.ones((3,3), np.uint8)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_CLOSE, kernel)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_OPEN, kernel)

        # Define audience areas (you might want to adjust this based on your video)
        # More conservative approach: only target very bright, saturated colors
        lower_audience = np.array([0, 100, 150])  # More restrictive
        upper_audience = np.array([180, 255, 255])
        audience_mask = cv2.inRange(hsv, lower_audience, upper_audience)
        
        # Remove ball pixels from audience mask to prevent ball deletion
        audience_mask = cv2.bitwise_and(audience_mask, cv2.bitwise_not(ball_mask))

        # Combine grass and audience masks
        combined_mask = cv2.bitwise_or(grass_mask, audience_mask)

        # Create the final mask: remove grass and audience but keep the ball
        # Invert the combined mask to keep the field and ball
        keep_mask = cv2.bitwise_not(combined_mask)
        
        # Ensure ball pixels are always kept (add ball mask to keep_mask)
        keep_mask = cv2.bitwise_or(keep_mask, ball_mask)

        # Apply the mask to the grayscale frame
        processed_frame = cv2.bitwise_and(gray_frame, gray_frame, mask=keep_mask)

        # Write the processed frame to the output video
        out.write(processed_frame)
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Processed video saved as: {output_path}")

def process_video_alternative(input_path, output_path):
    """
    Alternative approach: Use field detection instead of grass removal
    """
    cap = cv2.VideoCapture(input_path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)
    
    print(f"Processing {total_frames} frames...")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define what to keep (field area + ball)
        # Keep the playing field (moderate green) and the ball
        lower_field = np.array([35, 30, 30])
        upper_field = np.array([85, 255, 200])
        field_mask = cv2.inRange(hsv, lower_field, upper_field)
        
        # Ball detection (white/bright objects)
        lower_ball = np.array([0, 0, 180])
        upper_ball = np.array([180, 40, 255])
        ball_mask = cv2.inRange(hsv, lower_ball, upper_ball)
        
        # Keep field and ball
        keep_mask = cv2.bitwise_or(field_mask, ball_mask)
        
        # Apply morphological operations to clean up
        kernel = np.ones((3,3), np.uint8)
        keep_mask = cv2.morphologyEx(keep_mask, cv2.MORPH_CLOSE, kernel)
        
        # Apply the mask
        processed_frame = cv2.bitwise_and(gray_frame, gray_frame, mask=keep_mask)
        
        out.write(processed_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Processed video saved as: {output_path}")

# Paths
input_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv"
output_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT_processed2.mp4"

# Check if input video exists
if os.path.exists(input_video):
    # Try the main approach first
    process_video(input_video, output_video)
    
    # If you want to try the alternative approach, uncomment below:
    output_video_alt = output_video.replace('.mp4', '_alt.mp4')
    process_video_alternative(input_video, output_video_alt)
else:
    print(f"Input video not found: {input_video}")

Processing 1801 frames...
Processed 100/1801 frames
Processed 200/1801 frames
Processed 300/1801 frames
Processed 400/1801 frames
Processed 500/1801 frames
Processed 600/1801 frames
Processed 700/1801 frames
Processed 800/1801 frames
Processed 900/1801 frames
Processed 1000/1801 frames
Processed 1100/1801 frames
Processed 1200/1801 frames
Processed 1300/1801 frames
Processed 1400/1801 frames
Processed 1500/1801 frames
Processed 1600/1801 frames
Processed 1700/1801 frames
Processed 1800/1801 frames
Processed video saved as: F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT_processed2.mp4
Processing 1801 frames...
Processed 100/1801 frames
Processed 200/1801 frames
Processed 300/1801 frames
Processed 400/1801 frames
Processed 500/1801 frames
Processed 600/1801 frames
Processed 700/1801 frames
Processed 800/1801 frames
Processed 900/1801 frames
Processed 1000/1801 frames
Processed 1100/1801 frames
Processed 1200/1801 frames
Processed 1300/1801 frames
Process

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import os

# -----------------------
# Mediapipe pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
# -----------------------

# Parameters for HSV grass masking (tweak as needed)
lower_green = np.array([35, 40, 40])
upper_green = np.array([85, 255, 255])

# Load video
video_path = "your_video.mp4"
cap = cv2.VideoCapture(video_path)

# Optical flow init
ret, prev_frame = cap.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

all_keypoints = []
all_flow = []

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ---------------------
    # 1. Background removal: zero out grass
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    frame[mask > 0] = 0

    # ---------------------
    # 2. Skeleton extraction
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    keypoints = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
    else:
        keypoints = [0] * (33 * 4)  # fill with zeros if no detection

    all_keypoints.append(keypoints)

    # ---------------------
    # 3. Optical flow calculation
    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, 
        None, 0.5, 3, 15, 3, 5, 1.2, 0)
    all_flow.append(flow.mean(axis=(0,1)))  # average flow vector per frame

    # Update prev frame
    prev_gray = gray.copy()
    frame_idx += 1

cap.release()
pose.close()

# ---------------------
# Save features to a single file
np.savez("video_features.npz", keypoints=np.array(all_keypoints), flow=np.array(all_flow))
print(f"Saved {frame_idx} frames of features.")


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [16]:
import cv2
import numpy as np
from moviepy import VideoFileClip
import os

def create_football_heatmap_video(input_path, output_path):
    """
    Process football video to remove background and create heatmaps for players, referees, ball, and goal.
    """
    # Load video
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Initialize background subtractor for grass detection
    backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
    
    # Accumulator for heatmap
    heatmap_accumulator = np.zeros((height, width), dtype=np.float32)
    frame_count = 0
    
    print(f"Processing {total_frames} frames...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")
        
        # Convert to HSV for better color segmentation
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Define green grass color range in HSV
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])
        grass_mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # Remove advertisement boards (typically at edges and bright colors)
        # Define ranges for bright colors (advertisements)
        lower_bright = np.array([0, 100, 100])
        upper_bright = np.array([180, 255, 255])
        bright_mask = cv2.inRange(hsv, lower_bright, upper_bright)
        
        # Create edge mask for advertisement removal
        edge_thickness = 50  # pixels from edge
        edge_mask = np.zeros((height, width), dtype=np.uint8)
        edge_mask[:edge_thickness, :] = 255  # Top edge
        edge_mask[-edge_thickness:, :] = 255  # Bottom edge
        edge_mask[:, :edge_thickness] = 255  # Left edge
        edge_mask[:, -edge_thickness:] = 255  # Right edge
        
        # Combine masks for background removal
        background_mask = cv2.bitwise_or(grass_mask, edge_mask)
        background_mask = cv2.bitwise_or(background_mask, bright_mask)
        
        # Apply morphological operations to clean up mask
        kernel = np.ones((5, 5), np.uint8)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_CLOSE, kernel)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_OPEN, kernel)
        
        # Create foreground mask (players, referees, ball, goal)
        foreground_mask = cv2.bitwise_not(background_mask)
        
        # Apply Gaussian blur to foreground mask for smoother heatmap
        foreground_mask_blur = cv2.GaussianBlur(foreground_mask, (15, 15), 0)
        
        # Accumulate heatmap data
        heatmap_accumulator += foreground_mask_blur.astype(np.float32) / 255.0
        
        # Create output frame
        output_frame = np.zeros_like(frame)
        
        # Normalize current heatmap for visualization
        current_heatmap = heatmap_accumulator / frame_count
        current_heatmap_normalized = (current_heatmap * 255).astype(np.uint8)
        
        # Apply colormap to heatmap
        heatmap_colored = cv2.applyColorMap(current_heatmap_normalized, cv2.COLORMAP_JET)
        
        # Combine with foreground mask to show only active areas
        foreground_3channel = cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR)
        output_frame = cv2.bitwise_and(heatmap_colored, foreground_3channel)
        
        # Write frame
        out.write(output_frame)
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Heatmap video saved as: {output_path}")

# Paths
input_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv"
output_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT_heatmap.mp4"
output_video_advanced = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT_heatmap_advanced.mp4"

# Check if input video exists
if os.path.exists(input_video):
    print("Processing basic heatmap...")
    create_football_heatmap_video(input_video, output_video)

else:
    print(f"Input video not found: {input_video}")

Processing basic heatmap...
Processing 1801 frames...
Processed 100/1801 frames
Processed 200/1801 frames
Processed 300/1801 frames
Processed 400/1801 frames
Processed 500/1801 frames
Processed 600/1801 frames
Processed 700/1801 frames
Processed 800/1801 frames
Processed 900/1801 frames
Processed 1000/1801 frames
Processed 1100/1801 frames
Processed 1200/1801 frames
Processed 1300/1801 frames
Processed 1400/1801 frames
Processed 1500/1801 frames
Processed 1600/1801 frames
Processed 1700/1801 frames
Processed 1800/1801 frames
Heatmap video saved as: F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT_heatmap.mp4


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

lower_green = np.array([35, 40, 40])
upper_green = np.array([85, 255, 255])

cap = cv2.VideoCapture("your_video.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter("debug_output.mp4", fourcc, 30.0, 
                            (int(cap.get(3)), int(cap.get(4))))

ret, prev_frame = cap.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

all_keypoints = []
all_flow_mean = []
all_flow_full = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    frame[mask > 0] = 0

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    keypoints = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
        # Draw skeleton for debug video
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        keypoints = [0] * (33 * 4)

    all_keypoints.append(keypoints)

    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, 
        None, 0.5, 3, 15, 3, 5, 1.2, 0)
    all_flow_mean.append(flow.mean(axis=(0,1)))
    all_flow_full.append(flow)

    prev_gray = gray.copy()

    out_video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

cap.release()
pose.close()
out_video.release()

np.savez("video_features_full.npz", 
    keypoints=np.array(all_keypoints), 
    flow_mean=np.array(all_flow_mean),
    flow_full=np.array(all_flow_full))
print("Done. Saved debug video and full features.")


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

lower_green = np.array([35, 40, 40])
upper_green = np.array([85, 255, 255])

input_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv"
output_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\debug_output.mp4"

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(output_video, fourcc, 30.0, 
                            (int(cap.get(3)), int(cap.get(4))))

ret, prev_frame = cap.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

all_keypoints = []
all_flow_mean = []
all_flow_full = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    frame[mask > 0] = 0

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    keypoints = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
        # Draw skeleton for debug video
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        keypoints = [0] * (33 * 4)

    all_keypoints.append(keypoints)

    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, 
        None, 0.5, 3, 15, 3, 5, 1.2, 0)
    all_flow_mean.append(flow.mean(axis=(0,1)))
    all_flow_full.append(flow)

    prev_gray = gray.copy()

    out_video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

cap.release()
pose.close()
out_video.release()

np.savez(r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\video_features_full.npz", 
    keypoints=np.array(all_keypoints), 
    flow_mean=np.array(all_flow_mean),
    flow_full=np.array(all_flow_full))
print("Done. Saved debug video and full features.")


MemoryError: Unable to allocate 12.4 GiB for an array with shape (1799, 720, 1280, 2) and data type float32

In [5]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

lower_green = np.array([35, 40, 40])
upper_green = np.array([85, 255, 255])

input_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_720pT.mkv"
output_video = r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\debug_output.mp4"

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(output_video, fourcc, 30.0, 
                            (int(cap.get(3)), int(cap.get(4))))

ret, prev_frame = cap.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2RGB)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_RGB2GRAY)

all_keypoints = []
all_flow_mean = []
all_flow_full = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    frame[mask > 0] = 0

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)

    keypoints = []
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y, lm.z, lm.visibility])
        # Draw skeleton for debug video
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        keypoints = [0] * (33 * 4)

    all_keypoints.append(keypoints)

    gray = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, 
        None, 0.5, 3, 15, 3, 5, 1.2, 0)
    all_flow_mean.append(flow.mean(axis=(0,1)))
    all_flow_full.append(flow)

    prev_gray = gray.copy()

    out_video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

cap.release()
pose.close()
out_video.release()

np.savez(r"F:\AIM Lab\Experiment\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\video_features_full.npz", 
    keypoints=np.array(all_keypoints), 
    flow_mean=np.array(all_flow_mean))
print("Done. Saved debug video and full features.")


Done. Saved debug video and full features.
